In [1]:
import requests
import json
import time
import pandas as pd
import datetime

In [2]:
def make_query_chains(chains,function_str,date_from,date_to):
    query = '''query {{
        {function_str}(
    chains:[\"{chains}\"]
    from:\"{date_from}\"
    to:\"{date_to}\"
  )
  {{
    total_pokt
    total_relays
  }}

  
}}'''.format(
  function_str=function_str,
  chains=chains,
  date_from=date_from,
  date_to=date_to
)

    return query


def make_query(address,function_str,date_from,date_to):
    query = '''query {{
        {function_str}(
    addresses:[\"{address}\"]
    from:\"{date_from}\"
    to:\"{date_to}\"
  )
  {{
    total_tokens
    total_rewards
    total_relays
    total_fee
  }}

}}'''.format(
  function_str=function_str,
  date_from=date_from,
  date_to=date_to,
  address=address
)

    return query

In [ ]:
url='https://poktscan.com/api/graphql'
url_backup='https://fallback-api.poktscan.com/api/graphql' #needed for time when poktscan switched to v2
# urlv2='https://api.poktscan.com/poktscan/api/graphql' # in case you are on v2
headers = {'Authorization': '...', # enter poktscan API key
'Content-Type':'application/json'} 

### CSVs directly from poktscan.com/

In [2]:
#list of all nodes
df_node_list = pd.read_csv('pokt_nodes.csv')
df_node_list.loc[df_node_list['Stake Amount (POKT)']=='-','Stake Amount (POKT)']=0
df_node_list.loc[df_node_list['Stake From']==0,'Stake From']=1
df_node_list['datestr_unstake']=[ datetime.datetime.strptime(df_node_list['Unstaking Time'].iloc[i], '%b %d, %Y %H:%M %p') for i in range(0, len(df_node_list))]
df_node_list['datestr_unstake_year']=[ df_node_list.datestr_unstake.iloc[i].year for i in range(0, len(df_node_list))]
df_node_list.loc[df_node_list.datestr_unstake_year==1,'datestr_unstake']=datetime.datetime.strptime('2100-01-01','%Y-%m-%d')

/opt/homebrew/Caskroom/miniforge/base/envs/python38_playground/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3460: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [30]:
df_node_list.Address.nunique()

68696

In [9]:
#chain identifier
df_chains = pd.read_csv("csvs/pocket_chains.csv")

In [10]:
df_chains['chain_str']=[str(df_chains['RelayChainID'].iloc[i]) for i in range(0, len(df_chains))]

In [11]:
df_chains['chain_str_len'] = [len(df_chains.chain_str.iloc[i]) for i in range(0, len(df_chains))]
df_chains.loc[df_chains.chain_str_len==3,'chain_str'] = ["0"+df_chains[df_chains.chain_str_len==3].chain_str.iloc[i] for i in range(0, len(df_chains[df_chains.chain_str_len==3]))]
df_chains.loc[df_chains.chain_str_len==2,'chain_str'] = ["00"+df_chains[df_chains.chain_str_len==2].chain_str.iloc[i] for i in range(0, len(df_chains[df_chains.chain_str_len==2]))]
df_chains.loc[df_chains.chain_str_len==1,'chain_str'] = ["000"+df_chains[df_chains.chain_str_len==1].chain_str.iloc[i] for i in range(0, len(df_chains[df_chains.chain_str_len==1]))]

In [6]:
#blockheights to get start date of staking for nodes - also available via API
df_blocks = pd.read_csv('pocket_blocks.csv')

/opt/homebrew/Caskroom/miniforge/base/envs/python38_playground/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (2,5,6,7,10,13,14,16,18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
df_blocks['datestr']=[ datetime.datetime.strptime(df_blocks['Time'].iloc[i], '%b %d, %Y %H:%M %p') for i in range(0, len(df_blocks))]

In [8]:
df_node_list_all=df_node_list.merge(df_blocks[['Height','datestr']], how='left',left_on='Stake From',right_on='Height')

### API call for rewards data

In [3]:
function_str = 'getRewardsReport'
date_from='2020-07-01'
date_to='2023-05-31'


In [5]:
lst_address = df_node_list_all.Address.unique()

In [4]:
lst_month_start=[
    '2020-07-01',
'2020-08-01',
'2020-09-01',
'2020-10-01',
'2020-11-01',
'2020-12-01',
'2021-01-01',
'2021-02-01',
'2021-03-01',
'2021-04-01',
'2021-05-01',
'2021-06-01',
'2021-07-01',
'2021-08-01',
'2021-09-01',
'2021-10-01',
'2021-11-01',
'2021-12-01',
'2022-01-01',
'2022-02-01',
'2022-03-01',
'2022-04-01',
'2022-05-01',
'2022-06-01',
'2022-07-01',
'2022-08-01',
'2022-09-01',
'2022-10-01',
'2022-11-01',
'2022-12-01',
'2023-01-01',
'2023-02-01',
'2023-03-01',
'2023-04-01',
'2023-05-01'
]

In [5]:
lst_month_end = [
    '2020-07-31',
'2020-08-31',
'2020-09-30',
'2020-10-31',
'2020-11-30',
'2020-12-31',
'2021-01-31',
'2021-02-28',
'2021-03-31',
'2021-04-30',
'2021-05-31',
'2021-06-30',
'2021-07-31',
'2021-08-31',
'2021-09-30',
'2021-10-31',
'2021-11-30',
'2021-12-31',
'2022-01-31',
'2022-02-28',
'2022-03-31',
'2022-04-30',
'2022-05-31',
'2022-06-30',
'2022-07-31',
'2022-08-31',
'2022-09-30',
'2022-10-31',
'2022-11-30',
'2022-12-31',
'2023-01-31',
'2023-02-28',
'2023-03-31',
'2023-04-30',
'2023-05-31'
]

In [12]:
function_str="getRewardsByChain"
cnt_start=0

cnt = cnt_start
dct_out = {'chain':[],
'month':[],
'total_pokt':[],
'total_relays':[],
}

for cnt in range(0,df_chains.chain_str.nunique()):
    chain = df_chains.chain_str.iloc[cnt]
    print('chain: '+ chain)
    for num_month in range(32,len(lst_month_start)):
        print('month '+str(num_month))
        check=False
        attempt=0
        while (attempt<10) & (check==False):
            query = make_query_chains(chain,function_str,lst_month_start[num_month],lst_month_end[num_month])

            r = requests.post(url_backup, json={'query': query}, headers=headers)
            if r.status_code==200:
                result = r.json()['data']
                dct_out['chain'].append(chain)
                dct_out['month'].append(lst_month_start[num_month])
                for k in  result[function_str]:
                    dct_out[k].append(result[function_str][k])
                time.sleep(3)
                check=True

            elif r.status_code==429:
                time.sleep(60)
                attempt+=1
            else:
                print(r.status_code)
                time.sleep(0.2)
                attempt+=1
            
        if attempt>10:
                dct_out['chain'].append(chain)
                dct_out['month'].append(lst_month_start[num_month])
                dct_out['total_rewards'].append(0)
                dct_out['total_relays'].append(0)

        time.sleep(5)
        




chain: 0003
month 32
month 33
month 34
chain: 0004
month 32
month 33
month 34
chain: 0010
month 32
month 33
month 34
chain: 0048
month 32
month 33
month 34
chain: 03DF
month 32
month 33
month 34
chain: 0059
month 32
month 33
month 34
chain: 0021
month 32
month 33
month 34
chain: 0022
month 32
month 33
month 34
chain: 0028
month 32
month 33
month 34
chain: 0026
month 32
month 33
month 34
chain: 0024
month 32
month 33
month 34
chain: 0025
month 32
month 33
month 34
chain: 0023
month 32
month 33
month 34
chain: 0046
month 32
month 33
month 34
chain: 0049
month 32
month 33
month 34
chain: 0005
month 32
month 33
month 34
chain: 000A
month 32
month 33
month 34
chain: 0063
month 32
month 33
month 34
chain: 0027
month 32
month 33
month 34
chain: 000C
month 32
month 33
month 34
chain: 0040
month 32
month 33
month 34
chain: 0044
month 32
month 33
month 34
chain: 0056
month 32
month 33
month 34
chain: 0057
month 32
month 33
month 34
chain: 0058
month 32
month 33
month 34
chain: 0050
month 32
mont

In [16]:
# df_rewards_chains_monthly = pd.DataFrame(zip(dct_out['chain'],dct_out['month'],dct_out['total_pokt'],dct_out['total_relays']),columns=['chain','month','total_pokt','total_relays'])
df_tmp = pd.DataFrame.from_dict(dct_out)

In [14]:
df_tmp

,chain,month,total_pokt,total_relays
0,0003,2023-03-01,179514.7,375868972
1,0003,2023-04-01,216920.8,479731126
2,0003,2023-05-01,25002.6,57232482
3,0004,2023-03-01,1318074.2,2743567511
4,0004,2023-04-01,1233892.9,2725369333
...,...,...,...,...
112,0061,2023-04-01,0.0,0
113,0061,2023-05-01,0.0,0
114,03CB,2023-03-01,63943.0,89033418
115,03CB,2023-04-01,34961.0,50852024


In [17]:
df_rewards_chains_monthly=df_tmp

In [18]:
df_rewards_chains_monthly['is_active']=0
df_rewards_chains_monthly.loc[df_rewards_chains_monthly.total_pokt>0,'is_active']=1

In [19]:
df_rewards_monthly=df_rewards_chains_monthly.groupby('month').agg(
    num_chains = ('is_active','sum'),
    rewards=('total_pokt','sum'),
    relays=('total_relays','sum')
).reset_index()

In [20]:
df_rewards_monthly

,month,num_chains,rewards,relays
0,2023-03-01,36,17912154.7,36595621105
1,2023-04-01,35,16989383.1,37233317951
2,2023-05-01,33,1857603.4,4038103089


In [2]:
#below queries not used for rewards_data csv
#monthly rewards by node address

lst_address= df_node_list_all.Address.unique()

cnt = 0
cnt_address_start=0 #to allow for later start when error/restarts occurs
cnt_address=cnt_address_start
dct_out = {'address':[],
'month':[],
'total_tokens':[],
'total_rewards':[],
'total_relays':[],
'total_fee':[]
}
for a in lst_address[cnt_address_start:len(lst_address)]:
    if cnt_address%10==0:
        print(cnt_address)
    
    first_month= df_node_list_all[(df_node_list_all.Address==a)].datestr.iloc[0].month
    first_year= df_node_list_all[(df_node_list_all.Address==a)].datestr.iloc[0].year
    first = datetime.datetime.strptime(str(first_year) + '-' + str(first_month) + '-01','%Y-%m-%d')
    for num_month in range(0,len(lst_month_start)):
        dct_out['address'].append(a)
        dct_out['month'].append(lst_month_start[num_month])

        if (first<=datetime.datetime.strptime(lst_month_start[num_month],'%Y-%m-%d')) & (df_node_list_all[(df_node_list_all.Address==a)].datestr_unstake.iloc[0]>=datetime.datetime.strptime(lst_month_end[num_month],'%Y-%m-%d')):    
            query = make_query(a, function_str,lst_month_start[num_month],lst_month_end[num_month])
            # print(query)
            r = requests.post(url, json={'query': query}, headers=headers)
            if r.status_code==200:
                result = r.json()['data']
            elif r.status_code==429:
                time.sleep(60)
            else:
                print(cnt)
                print(r.status_code)
            for k in  result[function_str]:
                dct_out[k].append(result[function_str][k])

            time.sleep(3)
        else:
            dct_out['total_fee'].append(0)
            dct_out['total_rewards'].append(0)
            dct_out['total_relays'].append(0)
            dct_out['total_tokens'].append(0)
        
    cnt_address+=1
    time.sleep(10)
        




In [35]:
df_out=pd.DataFrame.from_dict(dct_out)

In [37]:
df_out=df_out.drop_duplicates()

In [38]:
df_out

,chain,month,total_pokt,total_relays
0,0003,2020-07-01,0.0,0
1,0003,2020-08-01,0.0,0
2,0003,2020-09-01,0.0,0
3,0003,2020-10-01,0.0,0
4,0003,2020-11-01,0.0,0
...,...,...,...,...
1243,0060,2023-02-01,101.3,202189
1244,0061,2023-01-01,949.4,1099654
1245,0061,2023-02-01,139.3,146144
1246,03CB,2023-01-01,30392.9,41097416


In [13]:
function_str = 'getRewardsReport'
date_from='2020-07-01'
date_to='2023-04-30'


In [16]:
#alltime rewards by node address
cnt = 0
cnt_address_start=0
cnt_address=cnt_address_start
dct_out = {'address':[],
'total_tokens':[],
'total_rewards':[],
'total_relays':[],
'total_fee':[]
}
for a in lst_address[cnt_address_start:len(lst_address)]:
    if cnt_address%50==0:
        print(cnt_address)
        time.sleep(3)
    
    dct_out['address'].append(a)

    query = make_query(a, function_str,date_from,date_to)
    # print(query)
    r = requests.post(url, json={'query': query}, headers=headers)
    if r.status_code==200:
        result = r.json()['data']
    elif r.status_code==429:
        time.sleep(60)
    else:
        print(cnt)
        print(r.status_code)
    for k in  result[function_str]:
        dct_out[k].append(result[function_str][k])
    
        
    cnt_address+=1
    time.sleep(0.4)
    
        




0
50
0
504
100
150
200
250
0
504
300
350
400
0
504
450
0
504
0
504
500
550
600
0
504
650
700
750
0
504
800
850
900
0
504
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
0
504
1450
0
504
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
0
504
2000
2050
2100
0
504
2150
2200
2250
2300
0
504
0
504
2350
2400
2450
0
504
0
504
2500
2550
2600
2650
2700
2750
2800
0
504
2850
2900
2950
0
504
